<a href="https://colab.research.google.com/github/jaqueline-amorim/Selecao-de-noticias-relevantes-com-arvore-de-decisao/blob/main/Selecao_de_noticias_relevantes_com_arvore_de_decisao.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Seleção de notícias relevantes com Árvore de decisão**

Este trabalho teve por objetivo criar uma árvore de decisão que através de uma API (interface de programação de aplicações), encontra notícias conforme o termo de busca, em fontes oficias pré determinadas, classifica os artigos, e os considerados relevantes são apresentados com o titutulo, descrição, URL e fonte.

In [ ]:
import requests
from sklearn.tree import DecisionTreeClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import classification_report

In [ ]:
# Sua chave de API do News API
api_key = 'coloque a sua chave aqui'

In [ ]:
# Termo de busca
query = 'Enchentes Rio Grande do Sul 2024', 'municípios afetados pelas enchentes Rio Grande do Sul 2024'

In [ ]:
# Lista de domínios de fontes oficiais (exemplos)
official_sources = [
    'globo.com', 'estadao.com.br', 'folha.uol.com.br', 'g1.globo.com',
    'agenciabrasil.ebc.com.br', 'bbc.com', 'brasil.mapbiomas.org',
    'estado.rs.gov.br', 'ufrgs.br'
]

In [ ]:
# Função para buscar notícias usando News API
def search_news_newsapi(query):
    url = f'https://newsapi.org/v2/everything'
    params = {
        'apiKey': api_key,
        'q': query,
        'language': 'pt',
        'domains': ','.join(official_sources),
        'pageSize': 50
    }
    response = requests.get(url, params=params)
    if response.status_code == 200:
        return response.json()
    else:
        print(f'Erro na requisição para {query}:', response.status_code)
        return None

In [ ]:
# Exemplo de uso: coleta de dados
data = search_news_newsapi(query)

if data:
    if 'articles' in data:
        articles = data['articles']
        X = [article['title'] + ' ' + article['description'] for article in articles if 'description' in article]
        y = [1 if 'enchentes' in article['title'].lower() or 'enchentes' in article['description'].lower() else 0 for article in articles]

        # Dividir dados em treino e teste
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

        # Criar pipeline com vetorização de texto e árvore de decisão
        pipeline = Pipeline([
            ('vectorizer', CountVectorizer()),  # Vetorização de texto simples
            ('classifier', DecisionTreeClassifier(random_state=42))  # Árvore de decisão com seed para reproducibilidade
        ])

        # Treinar o modelo
        pipeline.fit(X_train, y_train)

        # Avaliar o modelo
        y_pred = pipeline.predict(X_test)
        print(classification_report(y_test, y_pred))

        # Avaliação com validação cruzada
        scores = cross_val_score(pipeline, X, y, cv=5)  # cv=5 para 5-fold cross-validation
        print(f'Acurácia da validação cruzada: {scores.mean():.2f} (+/- {scores.std() * 2:.2f})')

        # Treinar o modelo com todos os dados
        pipeline.fit(X, y)

        # Exemplo de uso da árvore de decisão treinada para classificar novos artigos
        new_articles = [
            "enchentes",
            "combate às enchentes",
            "impacto das enchentes"
        ]
        new_articles_formatted = [' '.join(new_article.split()) for new_article in new_articles]  # Formatando os novos artigos

        predictions = pipeline.predict(new_articles_formatted)

        # Exibir resultados
        for article, prediction in zip(new_articles, predictions):
            if prediction == 1:
                print(f'Artigo relevante: {article}')
                for a in articles:
                    # Verifica se qualquer palavra-chave do novo artigo está contida no título ou descrição do artigo da API
                    if any(keyword.lower() in (a['title'] + ' ' + a['description']).lower() for keyword in new_articles):
                        print('Título:', a['title'])
                        print('Descrição:', a['description'] if 'description' in a else 'N/A')
                        print('URL:', a['url'] if 'url' in a else 'N/A')
                        print('Fonte:', a['source']['name'] if 'source' in a and 'name' in a['source'] else 'N/A')
                        print('-' * 200)
            else:
                print(f'Artigo não relevante: {article}')
    else:
        print('Nenhum artigo encontrado.')
else:
    print('Erro ao obter dados da API.')

